# Post-process morphotope IDs to ensure IDs allocated to noise are unique per contiguity group

In [1]:
import geopandas as gpd

from libpysal import graph

In [6]:
v = "v10"

country = "fr_sp_nl_be"
regions_datadir = "/data/uscuni-ulce/"
region_hulls = gpd.read_parquet(
    regions_datadir + "regions/" + f"{country}_regions_hull.parquet"
)


In [8]:
%%time

for region in region_hulls.index:
    df = gpd.read_parquet(
        f"/data/uscuni-ulce/processed_data/clusters/clusters_{region}_{v}.pq"
    )
    w = graph.read_parquet(
        f"/data/uscuni-ulce/processed_data/neigh_graphs/tessellation_graph_{region}.parquet"
    )
    noise = df[df.morph.str.endswith("-1")]
    noise_labels = w.subgraph(noise.index).component_labels
    df["unique_morph"] = df["morph"]
    df.loc[noise.index, "unique_morph"] = df.loc[noise.index, "unique_morph"].str[
        :-1
    ] + (noise_labels + 1).astype("str")
    df.to_parquet(f"/data/uscuni-ulce/processed_data/clusters/clusters_{region}_{v}.pq")

CPU times: user 5min 11s, sys: 31.7 s, total: 5min 43s
Wall time: 6min 5s
